# Game Playing with RL: Teaching AI to Master Games

From classic Atari to board games, RL has revolutionized game AI!

## What You'll Learn

By the end of this notebook, you'll understand:
- The arcade cabinet analogy: how RL learns to play games
- Classic control environments (CartPole, LunarLander, etc.)
- Training agents with Stable-Baselines3
- Evaluating and visualizing game performance
- Hyperparameter tuning for different games
- The journey from simple games to superhuman AI

**Prerequisites:** Policy gradient and PPO notebooks

**Time:** ~35 minutes

---
## The Big Picture: The Arcade Cabinet Analogy

```
    ┌────────────────────────────────────────────────────────────────┐
    │          THE ARCADE CABINET ANALOGY                            │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  Imagine teaching someone to play arcade games...             │
    │                                                                │
    │  THE SCREEN (State):                                          │
    │    What the player sees                                       │
    │    Position of characters, obstacles, enemies                 │
    │                                                                │
    │  THE JOYSTICK (Actions):                                      │
    │    Discrete: Up, Down, Left, Right, Fire                     │
    │    Or: Jump, Duck, Attack                                    │
    │                                                                │
    │  THE SCORE (Reward):                                          │
    │    Points for collecting items                               │
    │    Penalties for losing lives                                │
    │    Bonus for completing levels                               │
    │                                                                │
    │  THE PLAYER (RL Agent):                                       │
    │    Learns by trial and error                                 │
    │    Gets better with practice                                 │
    │    Develops strategies and reflexes                          │
    │                                                                │
    │  LEARNING PROCESS:                                            │
    │    Play game → See score → Adjust strategy → Repeat!        │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import FancyBboxPatch, Circle, Rectangle
import warnings
warnings.filterwarnings('ignore')

# Check gymnasium availability
try:
    import gymnasium as gym
    GYM_AVAILABLE = True
    print("✓ Gymnasium is installed!")
except ImportError:
    try:
        import gym
        GYM_AVAILABLE = True
        print("✓ OpenAI Gym is installed (legacy version)")
    except ImportError:
        GYM_AVAILABLE = False
        print("✗ Gymnasium not installed.")
        print("  Install with: pip install gymnasium")

# Check stable-baselines3
try:
    from stable_baselines3 import PPO, DQN, A2C
    from stable_baselines3.common.evaluation import evaluate_policy
    SB3_AVAILABLE = True
    print("✓ Stable-Baselines3 is installed!")
except ImportError:
    SB3_AVAILABLE = False
    print("✗ Stable-Baselines3 not installed.")
    print("  Install with: pip install stable-baselines3")

In [ ]:
# Visualize the RL game playing concept

fig, ax = plt.subplots(figsize=(14, 10))
ax.set_xlim(0, 14)
ax.set_ylim(0, 12)
ax.axis('off')
ax.set_title('RL for Game Playing: The Learning Loop', fontsize=16, fontweight='bold')

# Game screen (environment)
screen_box = FancyBboxPatch((0.5, 6), 5, 5, boxstyle="round,pad=0.1",
                             facecolor='#e3f2fd', edgecolor='#1976d2', linewidth=3)
ax.add_patch(screen_box)
ax.text(3, 10.3, 'GAME ENVIRONMENT', ha='center', fontsize=11, fontweight='bold', color='#1976d2')
ax.text(3, 9.5, '(CartPole, LunarLander, etc.)', ha='center', fontsize=9)

# Draw a simple game scene
# Ground
ax.plot([1, 5], [6.8, 6.8], 'brown', linewidth=2)
# Cart
cart = Rectangle((2.5, 6.9), 1, 0.3, facecolor='#4caf50', edgecolor='black')
ax.add_patch(cart)
# Pole
ax.plot([3, 2.5], [7.2, 8.5], 'brown', linewidth=4)
# Wheels
wheel1 = Circle((2.7, 6.85), 0.15, facecolor='black')
wheel2 = Circle((3.3, 6.85), 0.15, facecolor='black')
ax.add_patch(wheel1)
ax.add_patch(wheel2)

# Agent (neural network)
agent_box = FancyBboxPatch((8, 6.5), 5, 4, boxstyle="round,pad=0.1",
                            facecolor='#c8e6c9', edgecolor='#388e3c', linewidth=3)
ax.add_patch(agent_box)
ax.text(10.5, 9.8, 'RL AGENT', ha='center', fontsize=11, fontweight='bold', color='#388e3c')
ax.text(10.5, 9.1, '(Neural Network Policy)', ha='center', fontsize=9)

# Draw simple neural net
for i in range(3):
    c = Circle((9, 7.5 + i*0.5), 0.15, facecolor='#388e3c', edgecolor='black')
    ax.add_patch(c)
for i in range(4):
    c = Circle((10.5, 7.25 + i*0.5), 0.15, facecolor='#388e3c', edgecolor='black')
    ax.add_patch(c)
for i in range(2):
    c = Circle((12, 7.75 + i*0.5), 0.15, facecolor='#388e3c', edgecolor='black')
    ax.add_patch(c)

# State arrow
ax.annotate('', xy=(7.9, 9), xytext=(5.6, 9),
            arrowprops=dict(arrowstyle='->', lw=3, color='#1976d2'))
ax.text(6.75, 9.5, 'State\n(observation)', ha='center', fontsize=9, color='#1976d2')

# Action arrow
ax.annotate('', xy=(5.6, 7), xytext=(7.9, 7),
            arrowprops=dict(arrowstyle='->', lw=3, color='#388e3c'))
ax.text(6.75, 7.5, 'Action\n(left/right)', ha='center', fontsize=9, color='#388e3c')

# Reward box
reward_box = FancyBboxPatch((5.5, 3), 3, 2, boxstyle="round,pad=0.1",
                             facecolor='#fff3e0', edgecolor='#f57c00', linewidth=3)
ax.add_patch(reward_box)
ax.text(7, 4.3, 'REWARD', ha='center', fontsize=11, fontweight='bold', color='#f57c00')
ax.text(7, 3.5, '+1 for balance\n-10 for falling', ha='center', fontsize=9)

# Reward arrows
ax.annotate('', xy=(5.5, 4), xytext=(3, 5.9),
            arrowprops=dict(arrowstyle='->', lw=2, color='#f57c00'))
ax.annotate('', xy=(10.5, 6.4), xytext=(8.5, 5.1),
            arrowprops=dict(arrowstyle='->', lw=2, color='#f57c00'))

# Learning update
ax.text(7, 1.5, 'Agent learns: "Actions that maximize reward are best!"',
        ha='center', fontsize=11, style='italic', color='#666')

plt.tight_layout()
plt.show()

print("\nRL GAME PLAYING LOOP:")
print("  1. Environment sends state (what agent sees)")
print("  2. Agent chooses action based on policy")
print("  3. Environment returns reward and next state")
print("  4. Agent updates policy to get more reward")
print("  5. Repeat until agent masters the game!")

---
## Classic Control Environments

```
    ┌────────────────────────────────────────────────────────────────┐
    │              CLASSIC CONTROL ENVIRONMENTS                      │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  CARTPOLE (Difficulty: ⭐)                                     │
    │    Task: Balance a pole on a moving cart                     │
    │    State: Cart position, velocity, pole angle, angular vel   │
    │    Actions: Push left (0) or right (1)                       │
    │    Reward: +1 for each timestep pole stays balanced          │
    │    Success: 500 timesteps (solved in ~100 episodes)          │
    │                                                                │
    │  LUNARLANDER (Difficulty: ⭐⭐)                                │
    │    Task: Land a spacecraft safely between flags              │
    │    State: Position, velocity, angle, leg contact             │
    │    Actions: Nothing, fire left, fire main, fire right        │
    │    Reward: +100 to +140 for landing, -100 for crash         │
    │    Success: Average reward > 200 (solved in ~500 episodes)   │
    │                                                                │
    │  MOUNTAINCAR (Difficulty: ⭐⭐⭐)                              │
    │    Task: Drive car up a steep hill                           │
    │    State: Position, velocity                                 │
    │    Actions: Push left, nothing, push right                   │
    │    Reward: -1 each step (sparse reward - hard!)              │
    │    Challenge: Car can't climb directly - must gain momentum  │
    │                                                                │
    │  ACROBOT (Difficulty: ⭐⭐⭐)                                  │
    │    Task: Swing double pendulum to reach target               │
    │    State: Joint angles and angular velocities                │
    │    Actions: Torque on middle joint                           │
    │    Reward: -1 until target reached (sparse reward)           │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Explore available environments

print("CLASSIC CONTROL ENVIRONMENTS")
print("="*70)

if GYM_AVAILABLE:
    environments = {
        'CartPole-v1': {
            'description': 'Balance a pole on a cart',
            'difficulty': '⭐',
            'timesteps_to_solve': '~50K',
            'best_algorithm': 'DQN, PPO, A2C',
        },
        'LunarLander-v2': {
            'description': 'Land a spacecraft safely',
            'difficulty': '⭐⭐',
            'timesteps_to_solve': '~500K',
            'best_algorithm': 'PPO, DQN',
        },
        'MountainCar-v0': {
            'description': 'Drive car up a hill',
            'difficulty': '⭐⭐⭐',
            'timesteps_to_solve': '~200K',
            'best_algorithm': 'DQN with exploration bonus',
        },
        'Acrobot-v1': {
            'description': 'Swing double pendulum',
            'difficulty': '⭐⭐⭐',
            'timesteps_to_solve': '~100K',
            'best_algorithm': 'A2C, PPO',
        },
    }
    
    for env_name, info in environments.items():
        print(f"\n{env_name}")
        print(f"  Description: {info['description']}")
        print(f"  Difficulty: {info['difficulty']}")
        print(f"  Timesteps to solve: {info['timesteps_to_solve']}")
        print(f"  Best algorithms: {info['best_algorithm']}")
        
        try:
            env = gym.make(env_name)
            print(f"  Observation space: {env.observation_space}")
            print(f"  Action space: {env.action_space}")
            env.close()
        except Exception as e:
            print(f"  (Environment not available: {e})")
else:
    print("Install gymnasium to explore environments!")

print("\n" + "="*70)

In [ ]:
# Visualize environment comparison

fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Environment data
env_data = [
    ('CartPole', 4, 2, 'Easy', '#4caf50'),
    ('LunarLander', 8, 4, 'Medium', '#ff9800'),
    ('MountainCar', 2, 3, 'Hard (sparse)', '#f44336'),
    ('Acrobot', 6, 3, 'Hard (sparse)', '#f44336'),
]

# Top left: State space comparison
ax1 = axes[0, 0]
names = [e[0] for e in env_data]
state_dims = [e[1] for e in env_data]
colors = [e[4] for e in env_data]
bars = ax1.bar(names, state_dims, color=colors, edgecolor='black', linewidth=2)
ax1.set_ylabel('State Dimensions', fontsize=11)
ax1.set_title('State Space Complexity', fontsize=12, fontweight='bold')
ax1.grid(True, alpha=0.3, axis='y')
for bar, dim in zip(bars, state_dims):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.2, 
             str(dim), ha='center', fontsize=10, fontweight='bold')

# Top right: Action space comparison
ax2 = axes[0, 1]
action_dims = [e[2] for e in env_data]
bars = ax2.bar(names, action_dims, color=colors, edgecolor='black', linewidth=2)
ax2.set_ylabel('Number of Actions', fontsize=11)
ax2.set_title('Action Space Size', fontsize=12, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='y')
for bar, dim in zip(bars, action_dims):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
             str(dim), ha='center', fontsize=10, fontweight='bold')

# Bottom left: Training time comparison
ax3 = axes[1, 0]
timesteps = [50, 500, 200, 100]  # in thousands
bars = ax3.bar(names, timesteps, color=colors, edgecolor='black', linewidth=2)
ax3.set_ylabel('Timesteps (thousands)', fontsize=11)
ax3.set_title('Typical Training Time to Solve', fontsize=12, fontweight='bold')
ax3.grid(True, alpha=0.3, axis='y')
for bar, ts in zip(bars, timesteps):
    ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 15,
             f'{ts}K', ha='center', fontsize=10, fontweight='bold')

# Bottom right: Difficulty factors
ax4 = axes[1, 1]
ax4.axis('off')
ax4.set_title('Difficulty Breakdown', fontsize=12, fontweight='bold')

difficulty_text = """
WHAT MAKES GAMES HARD FOR RL:

• Sparse Rewards (MountainCar, Acrobot)
  - Only get reward at the end
  - Agent must explore without guidance
  - Solution: Curiosity, reward shaping

• Large State Space (LunarLander)
  - More dimensions = harder to learn
  - Need more samples to cover space
  - Solution: Function approximation

• Credit Assignment
  - Which action caused the reward?
  - Delayed consequences
  - Solution: Value functions, GAE
"""
ax4.text(0.1, 0.9, difficulty_text, transform=ax4.transAxes, fontsize=10,
         verticalalignment='top', family='monospace')

plt.tight_layout()
plt.show()

---
## Playing Games with Random Policy

Let's first see how a random agent performs!

In [ ]:
def play_random_episodes(env_name, n_episodes=10):
    """
    Play episodes with random actions.
    
    This establishes a baseline - how well do we do
    with no learning at all?
    """
    if not GYM_AVAILABLE:
        print("Gymnasium not available")
        return None
    
    try:
        env = gym.make(env_name)
    except:
        print(f"Environment {env_name} not available")
        return None
    
    episode_rewards = []
    episode_lengths = []
    
    for ep in range(n_episodes):
        obs, _ = env.reset()
        total_reward = 0
        steps = 0
        done = False
        
        while not done:
            # Random action!
            action = env.action_space.sample()
            obs, reward, terminated, truncated, _ = env.step(action)
            done = terminated or truncated
            total_reward += reward
            steps += 1
        
        episode_rewards.append(total_reward)
        episode_lengths.append(steps)
    
    env.close()
    
    return {
        'rewards': episode_rewards,
        'lengths': episode_lengths,
        'mean_reward': np.mean(episode_rewards),
        'std_reward': np.std(episode_rewards),
        'mean_length': np.mean(episode_lengths),
    }


# Test random policy on different environments
print("RANDOM POLICY BASELINE")
print("="*60)

if GYM_AVAILABLE:
    test_envs = ['CartPole-v1', 'MountainCar-v0', 'Acrobot-v1']
    baselines = {}
    
    for env_name in test_envs:
        result = play_random_episodes(env_name, n_episodes=20)
        if result:
            baselines[env_name] = result
            print(f"\n{env_name}:")
            print(f"  Mean reward: {result['mean_reward']:.2f} ± {result['std_reward']:.2f}")
            print(f"  Mean episode length: {result['mean_length']:.1f}")
    
    print("\n" + "="*60)
    print("\nRANDOM POLICY IS BAD! Let's train RL agents...")
else:
    print("Install gymnasium to run experiments")

---
## Training with Stable-Baselines3

```
    ┌────────────────────────────────────────────────────────────────┐
    │              STABLE-BASELINES3 WORKFLOW                        │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  STEP 1: Create Environment                                   │
    │    env = gym.make('CartPole-v1')                             │
    │                                                                │
    │  STEP 2: Choose Algorithm                                     │
    │    • DQN - Good for discrete actions, sample efficient       │
    │    • PPO - Stable, works for most problems                   │
    │    • A2C - Fast training, good for simple environments       │
    │                                                                │
    │  STEP 3: Create Model                                         │
    │    model = PPO('MlpPolicy', env, verbose=1)                  │
    │                                                                │
    │  STEP 4: Train!                                               │
    │    model.learn(total_timesteps=100_000)                      │
    │                                                                │
    │  STEP 5: Evaluate                                             │
    │    mean_reward, std = evaluate_policy(model, env)            │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Complete training example

print("TRAINING RL AGENT ON CARTPOLE")
print("="*60)

if SB3_AVAILABLE and GYM_AVAILABLE:
    # Create environment
    env = gym.make('CartPole-v1')
    
    print("\n1. Environment created")
    print(f"   Observation space: {env.observation_space}")
    print(f"   Action space: {env.action_space}")
    
    # Create PPO model
    model = PPO(
        'MlpPolicy',           # Neural network policy
        env,                   # Environment
        learning_rate=3e-4,    # Learning rate
        n_steps=2048,          # Steps per update
        batch_size=64,         # Minibatch size
        n_epochs=10,           # Epochs per update
        gamma=0.99,            # Discount factor
        verbose=0,             # Quiet mode
    )
    
    print("\n2. PPO model created")
    print(f"   Policy: {model.policy}")
    
    # Evaluate before training
    print("\n3. Evaluating BEFORE training...")
    mean_reward_before, std_before = evaluate_policy(model, env, n_eval_episodes=10)
    print(f"   Mean reward: {mean_reward_before:.2f} ± {std_before:.2f}")
    
    # Train!
    print("\n4. Training for 50,000 timesteps...")
    model.learn(total_timesteps=50_000, progress_bar=True)
    
    # Evaluate after training
    print("\n5. Evaluating AFTER training...")
    mean_reward_after, std_after = evaluate_policy(model, env, n_eval_episodes=10)
    print(f"   Mean reward: {mean_reward_after:.2f} ± {std_after:.2f}")
    
    # Improvement
    improvement = mean_reward_after - mean_reward_before
    print(f"\n6. Improvement: +{improvement:.2f} reward!")
    
    if mean_reward_after >= 475:
        print("   🎉 Environment SOLVED! (threshold: 475)")
    
    env.close()
else:
    print("Install stable-baselines3 and gymnasium to train agents")

print("\n" + "="*60)

In [ ]:
# Compare different algorithms

def train_and_evaluate(algorithm_class, env_name, timesteps=30000, n_eval=10):
    """Train an algorithm and return results."""
    env = gym.make(env_name)
    
    # Create and train model
    model = algorithm_class('MlpPolicy', env, verbose=0)
    model.learn(total_timesteps=timesteps)
    
    # Evaluate
    mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=n_eval)
    
    env.close()
    return mean_reward, std_reward


print("ALGORITHM COMPARISON ON CARTPOLE")
print("="*60)

if SB3_AVAILABLE and GYM_AVAILABLE:
    algorithms = [
        ('PPO', PPO),
        ('A2C', A2C),
        ('DQN', DQN),
    ]
    
    results = {}
    for name, algo_class in algorithms:
        print(f"\nTraining {name}...")
        mean_r, std_r = train_and_evaluate(algo_class, 'CartPole-v1', timesteps=30000)
        results[name] = (mean_r, std_r)
        print(f"  Result: {mean_r:.2f} ± {std_r:.2f}")
    
    # Visualize comparison
    fig, ax = plt.subplots(figsize=(10, 6))
    
    names = list(results.keys())
    means = [results[n][0] for n in names]
    stds = [results[n][1] for n in names]
    colors = ['#4caf50', '#2196f3', '#ff9800']
    
    bars = ax.bar(names, means, yerr=stds, capsize=5, color=colors, 
                  edgecolor='black', linewidth=2)
    
    ax.axhline(y=475, color='red', linestyle='--', label='Solved threshold (475)')
    ax.axhline(y=results.get('Random', (25, 0))[0] if 'Random' in results else 25, 
               color='gray', linestyle=':', label='Random baseline (~25)')
    
    ax.set_ylabel('Mean Reward', fontsize=12)
    ax.set_title('Algorithm Comparison on CartPole-v1\n(30K training timesteps)', 
                 fontsize=14, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3, axis='y')
    ax.set_ylim(0, 520)
    
    for bar, mean in zip(bars, means):
        ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 20,
                f'{mean:.0f}', ha='center', fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
else:
    print("Install required libraries to compare algorithms")

---
## Hyperparameter Tuning

```
    ┌────────────────────────────────────────────────────────────────┐
    │              HYPERPARAMETER GUIDE                              │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  PPO HYPERPARAMETERS:                                         │
    │    learning_rate: Start with 3e-4, decrease if unstable      │
    │    n_steps: 2048 (how many steps before update)              │
    │    batch_size: 64-256 (minibatch for gradient)               │
    │    n_epochs: 10 (passes through collected data)              │
    │    gamma: 0.99 (discount factor - higher for long tasks)     │
    │    clip_range: 0.2 (PPO clipping - usually don't change)     │
    │                                                                │
    │  DQN HYPERPARAMETERS:                                         │
    │    learning_rate: 1e-4 to 1e-3                                │
    │    buffer_size: 100000+ (replay buffer)                      │
    │    batch_size: 32-128                                        │
    │    exploration_fraction: 0.1-0.2                             │
    │    target_update_interval: 1000-10000                        │
    │                                                                │
    │  GENERAL TIPS:                                                │
    │    • Start with defaults, then tune                          │
    │    • Learning rate is most important                         │
    │    • More timesteps usually helps                            │
    │    • Use multiple seeds for reliable results                 │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Hyperparameter tuning example

def tune_learning_rate(lr_values, env_name='CartPole-v1', timesteps=20000):
    """Compare different learning rates."""
    results = {}
    
    for lr in lr_values:
        env = gym.make(env_name)
        model = PPO('MlpPolicy', env, learning_rate=lr, verbose=0)
        model.learn(total_timesteps=timesteps)
        
        mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10)
        results[lr] = (mean_reward, std_reward)
        env.close()
        
        print(f"  LR={lr}: {mean_reward:.2f} ± {std_reward:.2f}")
    
    return results


print("LEARNING RATE TUNING")
print("="*60)

if SB3_AVAILABLE and GYM_AVAILABLE:
    lr_values = [1e-4, 3e-4, 1e-3, 3e-3]
    print(f"\nTesting learning rates: {lr_values}")
    print("\nTraining...")
    
    lr_results = tune_learning_rate(lr_values)
    
    # Visualize
    fig, ax = plt.subplots(figsize=(10, 6))
    
    lrs = list(lr_results.keys())
    means = [lr_results[lr][0] for lr in lrs]
    stds = [lr_results[lr][1] for lr in lrs]
    
    # Convert to log scale labels
    lr_labels = [f'{lr:.0e}' for lr in lrs]
    
    colors = ['#4caf50' if m > 400 else '#ff9800' if m > 200 else '#f44336' for m in means]
    bars = ax.bar(lr_labels, means, yerr=stds, capsize=5, color=colors,
                  edgecolor='black', linewidth=2)
    
    ax.axhline(y=475, color='green', linestyle='--', alpha=0.5, label='Solved')
    ax.set_xlabel('Learning Rate', fontsize=12)
    ax.set_ylabel('Mean Reward', fontsize=12)
    ax.set_title('Effect of Learning Rate on CartPole Performance', 
                 fontsize=14, fontweight='bold')
    ax.legend()
    ax.grid(True, alpha=0.3, axis='y')
    
    # Find best LR
    best_lr = lrs[np.argmax(means)]
    ax.text(0.5, 0.95, f'Best LR: {best_lr:.0e}', transform=ax.transAxes,
            fontsize=12, fontweight='bold', color='#388e3c',
            verticalalignment='top', horizontalalignment='center')
    
    plt.tight_layout()
    plt.show()
else:
    print("Install required libraries to tune hyperparameters")

---
## From Games to Superhuman AI: The Journey

```
    ┌────────────────────────────────────────────────────────────────┐
    │              HISTORY OF RL IN GAMES                            │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  1992: TD-GAMMON                                              │
    │    • Backgammon at expert level                               │
    │    • Temporal difference learning                             │
    │    • First neural network + RL success                       │
    │                                                                │
    │  2013: ATARI DQN (DeepMind)                                   │
    │    • Played 49 Atari games from pixels                       │
    │    • Superhuman on many games                                │
    │    • Deep RL revolution begins                               │
    │                                                                │
    │  2016: ALPHAGO                                                │
    │    • Beat world champion at Go                               │
    │    • Monte Carlo tree search + deep RL                       │
    │    • "Moment that changed AI"                                │
    │                                                                │
    │  2019: ALPHASTAR                                              │
    │    • Grandmaster level at StarCraft II                       │
    │    • Complex strategy and real-time control                  │
    │                                                                │
    │  2024: CLAUDE, GPT (RLHF)                                     │
    │    • Language models aligned with RL                         │
    │    • Same PPO algorithm, different domain                    │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Visualize RL history in games

fig, ax = plt.subplots(figsize=(14, 8))
ax.set_xlim(1990, 2026)
ax.set_ylim(0, 10)
ax.axis('off')
ax.set_title('Evolution of RL in Games', fontsize=16, fontweight='bold')

# Timeline
ax.axhline(y=5, xmin=0.02, xmax=0.98, color='#333', linewidth=3)

# Milestones
milestones = [
    (1992, 'TD-Gammon', 'Backgammon\nExpert level', '#64b5f6'),
    (2013, 'Atari DQN', '49 Atari games\nfrom pixels', '#81c784'),
    (2016, 'AlphaGo', 'Beat world\nGo champion', '#ffb74d'),
    (2019, 'AlphaStar', 'Grandmaster\nStarCraft II', '#ef5350'),
    (2024, 'RLHF LLMs', 'ChatGPT, Claude\naligned with RL', '#ce93d8'),
]

for i, (year, name, desc, color) in enumerate(milestones):
    # Circle on timeline
    circle = Circle((year, 5), 0.4, facecolor=color, edgecolor='black', linewidth=2)
    ax.add_patch(circle)
    
    # Alternate above/below
    y_offset = 2.5 if i % 2 == 0 else -2.5
    text_y = 5 + y_offset
    
    # Connector line
    ax.plot([year, year], [5, text_y + (0.8 if y_offset > 0 else -0.8)], 
            color=color, linewidth=2)
    
    # Text box
    box = FancyBboxPatch((year-2.5, text_y - 0.8 if y_offset > 0 else text_y + 0.3), 
                          5, 1.6, boxstyle="round,pad=0.1",
                          facecolor=color, edgecolor='black', alpha=0.8)
    ax.add_patch(box)
    
    ax.text(year, text_y + 0.4 if y_offset > 0 else text_y + 1.0, 
            f'{year}: {name}', ha='center', fontsize=10, fontweight='bold')
    ax.text(year, text_y - 0.3 if y_offset > 0 else text_y + 0.3,
            desc, ha='center', fontsize=9)

# Add complexity arrow
ax.annotate('', xy=(2025, 1), xytext=(1992, 1),
            arrowprops=dict(arrowstyle='->', lw=2, color='#666'))
ax.text(2008, 0.5, 'Increasing Complexity & Intelligence →', ha='center', fontsize=11, style='italic')

plt.tight_layout()
plt.show()

print("\nKEY INSIGHT:")
print("  The same core RL algorithms (value functions, policy gradients, PPO)")
print("  have scaled from simple games to superhuman AI in complex domains!")

---
## Summary: Game Playing with RL

### Key Environments

| Environment | Difficulty | Best Algorithm | Key Challenge |
|-------------|------------|----------------|---------------|
| CartPole | Easy | PPO, DQN, A2C | None (great for learning) |
| LunarLander | Medium | PPO, DQN | Fuel management |
| MountainCar | Hard | DQN + bonus | Sparse reward |
| Acrobot | Hard | A2C, PPO | Credit assignment |

### Algorithm Selection

| Algorithm | Pros | Cons | Use When |
|-----------|------|------|----------|
| **PPO** | Stable, works everywhere | Slower | Default choice |
| **DQN** | Sample efficient | Only discrete | Need efficiency |
| **A2C** | Fast training | Less stable | Quick prototyping |

### Quick Reference

```python
# The SB3 workflow
from stable_baselines3 import PPO
import gymnasium as gym

env = gym.make('CartPole-v1')
model = PPO('MlpPolicy', env, verbose=1)
model.learn(total_timesteps=50_000)
mean_reward, _ = evaluate_policy(model, env)
```

---
## Test Your Understanding

**1. What makes MountainCar harder than CartPole?**
<details>
<summary>Click to reveal answer</summary>
MountainCar has sparse rewards - the agent only gets reward (+0 instead of -1) when reaching the goal. This makes exploration very difficult since random actions never lead to success. The car must learn to build momentum by rocking back and forth, which requires long-horizon planning.
</details>

**2. When would you choose DQN over PPO?**
<details>
<summary>Click to reveal answer</summary>
DQN is more sample-efficient than PPO because it uses a replay buffer to reuse past experiences. Choose DQN when:
- You have discrete actions
- Environment interactions are expensive
- You need efficient use of data

Choose PPO when you need stability or have continuous actions.
</details>

**3. What is the 'solved' threshold and why does it matter?**
<details>
<summary>Click to reveal answer</summary>
The solved threshold is a benchmark score that indicates mastery of an environment. For CartPole, it's 475 (out of 500 maximum). It matters because:
- Provides an objective success criterion
- Allows comparison between algorithms
- Indicates when to stop training

Different environments have different thresholds based on their difficulty.
</details>

**4. Why is learning rate the most important hyperparameter?**
<details>
<summary>Click to reveal answer</summary>
Learning rate controls how much the policy changes with each update:
- Too high: Unstable training, oscillating performance
- Too low: Slow learning, may not converge in time
- Just right: Steady improvement toward optimal policy

Other hyperparameters matter, but a bad learning rate can completely prevent learning.
</details>

**5. How did RL scale from simple games to superhuman Go?**
<details>
<summary>Click to reveal answer</summary>
The same core principles scaled up:
1. Value functions → Neural network value approximation
2. Policy gradients → Deep policy networks
3. Exploration → Monte Carlo tree search
4. Self-play → Learning from playing against itself

Plus massive compute and clever engineering. AlphaGo used policy networks, value networks, and MCTS - all concepts we've covered!
</details>

---
## What's Next?

Game playing is just the beginning! In the next notebook, we'll explore **robotics simulation** - applying RL to continuous control problems like walking and manipulation.

**Continue to:** [Notebook 2: Robotics Simulation](02_robotics_simulation.ipynb)

---

*"From Pong to Go, RL has shown that learning to play is learning to think!"*